In [ ]:
from pathlib import Path
from dclpatch.main import PatchProject

In [ ]:
root_dir_path = Path('/mnt/c/Users/dsege/TEMP/Alexia/analysis_with_dcl_patch/')

In [ ]:
api = PatchProject(root_dir = root_dir_path)

In [ ]:
path_to_cell_recordings_dir = Path('/mnt/c/Users/dsege/TEMP/Alexia/NET-cre_vGat-flp/2021_12_09_C3/')

In [ ]:
api.add_cell_to_database(path_to_cell_recordings_dir = path_to_cell_recordings_dir, overwrite=True)

In [ ]:
api.database.cell_recordings_metadata

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [ ]:
global_cell_id = '0001'

In [ ]:
database = api.database

In [ ]:
filepaths = list(database.cell_recordings_metadata.loc[database.cell_recordings_metadata['global_cell_id'] == global_cell_id, 
                                                       'filepath_detected_events'].values)
filepaths = [Path(elem) for elem in filepaths]


In [ ]:
events_all_stim_paradigms = {'stimulation_string': list(), 
                            'amplitude': list(),
                            'inter_event_interval': list(),
                            'sweep': list()}



In [ ]:
for recording_filepath in filepaths:
    events_single_stim_paradigm = pd.read_csv(recording_filepath)
    columns = list(events_single_stim_paradigm.columns)
    for column_name in columns:
        if 'WaveN' in column_name:
            sweep_column = column_name
        elif 'PeakT' in column_name:
            event_time_column = column_name
        elif 'AmpY' in column_name:
            event_amplitude_column = column_name
        else:
            continue


    stim_string = database.cell_recordings_metadata.loc[database.cell_recordings_metadata['filepath_detected_events'] == recording_filepath.as_posix(),
                                                               'stimulation_string'].values[0]

    mean_inter_event_intervals = list()
    for sweep_id in events_single_stim_paradigm[sweep_column].unique():
        event_times = events_single_stim_paradigm.loc[events_single_stim_paradigm[sweep_column] == sweep_id, event_time_column]
        time_differences = event_times.rolling(2).apply(np.diff)
        mean_inter_event_intervals += list(time_differences.rolling(3, center=True).mean().values)


    events_all_stim_paradigms['stimulation_string'] += [stim_string] * events_single_stim_paradigm.shape[0]
    events_all_stim_paradigms['sweep'] += list(events_single_stim_paradigm[sweep_column].values)
    events_all_stim_paradigms['inter_event_interval'] += mean_inter_event_intervals
    events_all_stim_paradigms['amplitude'] += list(events_single_stim_paradigm[event_amplitude_column].values)

df_all_events = pd.DataFrame(data=events_all_stim_paradigms)

In [ ]:
parameters_to_compare = ['amplitude', 'inter_event_interval']

In [ ]:
stim_paradigms = list(df_all_events['stimulation_string'].unique())
stim_paradigms.remove('baseline')
n_multiple_comparisons = len(stim_paradigms)

fig = plt.figure(figsize=(15, 5*len(stim_paradigms)), facecolor='white')
gs = fig.add_gridspec(len(stim_paradigms), 2)

for stim_string in stim_paradigms:
    df_temp = df_all_events.loc[df_all_events['stimulation_string'].isin([stim_string, 'baseline'])]
    for parameter in parameters_to_compare:
            data_baseline = df_temp.loc[df_temp['stimulation_string'] == 'baseline', parameter].values
            data_stim_paradigm = df_temp.loc[df_temp['stimulation_string'] == stim_string, parameter].values
            test_statistic, uncorrected_p_value = stats.ks_2samp(data_baseline, data_stim_paradigm)
            corrected_p_value = uncorrected_p_value * n_multiple_comparisons
            if corrected_p_value <= 0.001:
                stars = '***'
            elif corrected_p_value <= 0.01:
                stars = '**'
            elif corrected_p_value <= 0.05:
                stars = '*'
            else:
                stars = 'n.s.'
                
            fig.add_subplot(gs[stim_paradigms.index(stim_string), parameters_to_compare.index(parameter)])
            sns.ecdfplot(data = df_temp, x = parameter, hue = 'stimulation_string', hue_order = ['baseline', stim_string])
            plt.title(f'{stim_string}: {stars} ({round(corrected_p_value, 3)})')
plt.suptitle(f'comparison of cell: {global_cell_id}')
plt.show()

In [ ]:
round(3.231, 3)

In [ ]:


for plane_index in range(z_dim):
    fig.add_subplot(gs[plane_index, 0])
    plt.imshow(preprocessed_zstack[plane_index])
    plt.ylabel(f'plane_{plane_index}', fontsize=14)
    if plane_index == 0:
        plt.title('input image', fontsize=14, pad=15)

In [ ]:
stim_string = stim_paradigms[0]
parameter = 'inter_event_interval'
n_multiple_comparisons = 2






In [ ]:
stars

In [ ]:
sns.ecdfplot(data=df_combo[df_combo['stimulation_string'] == 'baseline'], x='inter_event_interval', hue='stimulation_string')
#plt.xlim(-30, 0)

In [ ]:
stats.ks_2samp(data_baseline, data_stim_paradigm)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(sorted_data_baseline, p_baseline)
plt.plot(sorted_data_stim_paradigm, p_stim_paradigm)

In [ ]:
set(events_all_stim_paradigms['stimulation_string'])

In [ ]:
detected_events[event_amplitude_column]

In [ ]:
for sweep_id in detected_events[sweep_column].unique():
    print(detected_events.loc[detected_events[sweep_column] == sweep_id].shape[0])

In [ ]:
import seaborn as sns

In [ ]:
sns.ecdfplot(data=detected_events, x=event_amplitude_column)

In [ ]:
sns.swarmplot(data=detected_events, x=sweep_column, y=event_amplitude_column)

In [ ]:
detected_events

In [ ]:
animals = ['hund', 'katze', 'maus']
animals.remove('hund')

In [ ]:
animals

In [ ]:
api.database.recorded_cells

In [ ]:
overwrite = False

In [ ]:
if (path_to_cell_recordings_dir not in api.database.cells_in_database) or (overwrite == True):
    print('doing it')

# Code development

In [ ]:
import os
from pathlib import Path
from typing import Dict

import pandas as pd
import numpy as np

In [ ]:
def get_general_metadata_df(path_to_recordings_dir: Path) -> pd.DataFrame:
    metadata_df = pd.read_excel(path_to_recordings_dir.joinpath(f'{path_to_recordings_dir.name}.xlsx'),
                                sheet_name = 'General information')
    general_metadata = {'date': path_to_recordings_dir.name[:10],
                        'session_cell_id': path_to_recordings_dir.name[path_to_recordings_dir.name.rfind('_') + 1:],
                        'mouse_line': metadata_df['Animal line'][0],
                        'brain_region': metadata_df['Region'][0],
                        'cell_type': metadata_df['Type'][0],
                        'sex': metadata_df['Sex'][0]}
    return pd.DataFrame(general_metadata, index=[0])

In [ ]:
def get_stimulation_paradigms_df(path_to_recordings_dir: Path) -> pd.DataFrame:
    filepaths_stimulation_recordings = list()
    for elem in path_to_recordings_dir.iterdir():
        if 'datapoints' not in elem.name:
            if elem.name.endswith('.csv'):
                filepaths_stimulation_recordings.append(elem)
    stimulation_paradigms = {'stimulation_string': list(),
                             'stimulation_frequency-Hz': list(),
                             'stimulation_duration-ms': list(), 
                             'filepath_detected_events': list()}
    for filepath in filepaths_stimulation_recordings:
        filename = filepath.name
        filename = filename.replace('.csv', '')
        yyyy_mm_dd_ = filename[:11]
        filename = filename.replace(yyyy_mm_dd_, '')
        cell_id = filename[:filename.index('_')]
        filename = filename.replace(cell_id + '_', '')
        if 'Hz' in filename:
            stimulation_frequency = int(filename[:filename.index('Hz')])
            stimulation_duration = filename[filename.index('Hz') + 2 :]
            if 'ms' not in stimulation_duration:
                stimulation_duration = int(stimulation_duration[:stimulation_duration.find('s')]) * 1000
            else:
                stimulation_duration = int(stimulation_duration[:stimulation_duration.find('ms')])
            stimulation_paradigm = f'{stimulation_frequency}-Hz_for_{stimulation_duration}-ms'
        elif 'Bsl' in filename:
            stimulation_frequency = np.NaN
            stimulation_duration = np.NaN
            stimulation_paradigm = 'baseline'
        else:
            print(f'Warning: stimulation paradigm could not be identified for: {filepath.name}')
            stimulation_frequency = np.NaN
            stimulation_duration = np.NaN
            stimulation_paradigm = 'unknown'
        stimulation_paradigms['stimulation_string'].append(stimulation_paradigm)
        stimulation_paradigms['stimulation_frequency-Hz'].append(stimulation_frequency)
        stimulation_paradigms['stimulation_duration-ms'].append(stimulation_duration)
        stimulation_paradigms['filepath_detected_events'].append(filepath)
    return pd.DataFrame(stimulation_paradigms)

In [ ]:
global_cell_id = 1

In [ ]:
str(global_cell_id).zfill(4)

In [ ]:
def get
    general_metadata = get_general_metadata_df(cell_recording_dir)
    stimulation_paradigms = get_stimulation_paradigms_df(cell_recording_dir)
    stimulations_adjusted_general_metadata = pd.concat([general_metadata]*stimulation_paradigms.shape[0], ignore_index=True)
    recordings_overview = pd.concat([stimulations_adjusted_general_metadata, stimulation_paradigms], axis=1)
    recordings_overview['global_cell_id'] = str(global_cell_id).zfill(4)


In [ ]:
columns = ['global_cell_id', 'date', 'session_cell_id', 'mouse_line', 'sex', 'brain_region', 'cell_type', 
           'stimulation_string', 'stimulation_frequency-Hz', 'stimulation_duration-ms', 'filepath_detected_events']

In [ ]:
recordings_overview[columns]

In [ ]:
recordings_overview

In [ ]:
stimulation_paradigms

In [ ]:
df_cell = pd.DataFrame(recording_info)

In [ ]:
df_cell

In [ ]:
cell_recording_dir = Path('/mnt/c/Users/dsege/TEMP/Alexia/NET-cre_vGat-flp/2021_12_09_C2/')



metadata_df = pd.read_excel(cell_recording_dir.joinpath(f'{cell_recording_dir.name}.xlsx'),
                            sheet_name = 'General information')

mix_and_matchable_categories = dict()

general_metadata = {'mouse_line': metadata_df['Animal line'][0],
                    'brain_region': metadata_df['Region'][0],
                    'cell_type': metadata_df['Type'][0],
                    'sex': metadata_df['Sex'][0]}

filepaths_stimulation_recordings = list()
for elem in cell_recording_dir.iterdir():
    if 'datapoints' not in elem.name:
        if elem.name.endswith('.csv'):
            filepaths_stimulation_recordings.append(elem)


stimulation_paradigms = dict()
for filepath in filepaths_stimulation_recordings:
    filename = filepath.name
    filename = filename.replace('.csv', '')
    yyyy_mm_dd_ = filename[:11]
    filename = filename.replace(yyyy_mm_dd_, '')
    cell_id = filename[:filename.index('_')]
    filename = filename.replace(cell_id + '_', '')
    if 'Hz' in filename:
        stimulation_frequency = filename[:filename.index('Hz') + 2]
        stimulation_duration = filename[filename.index('Hz') + 2 :]
        stimulation_paradigm = f'{stimulation_frequency}_for_{stimulation_duration}'
    elif 'Bsl' in filename:
        stimulation_paradigm = 'baseline'
    else:
        print(f'Warning: stimulation paradigm could not be identified for: {filepath.name}')
        stimulation_paradigm = 'unknown'
    stimulation_paradigms[stimulation_paradigm] = filepath

In [ ]:
filepaths_stimulation_recordings

In [ ]:
stimulation_paradigms

In [ ]:
?filename.replace

In [ ]:
metadata_df['Animal line'][0]

In [ ]:
metadata_df

In [ ]:
df = pd.read_excel(cell_recording_dir.as_posix() + '/' + cell_recording_dir.name + '.xlsx', sheet_name = 'General information')
df

In [ ]:
df = pd.read_excel(cell_recording_dir.as_posix() + '/' + cell_recording_dir.name + '.xlsx', sheet_name = 'Recordings and cell properties', skiprows=2)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.columns

In [ ]:
dfs['General information']

In [ ]:
dfs['Recordings and cell properties']

In [ ]:
for elem in cell_recording_dir.iterdir():
    print(elem)

In [ ]:
pd.read_csv(filepath)

In [ ]:
path = Path('/mnt/c/Users/dsege/TEMP/Alexia/')

In [ ]:
for elem in path.iterdir():
    print(elem)

In [ ]:
for elem in path.glob(pattern='*'):
    print(elem)

In [ ]:
path.parts

In [ ]:
database = Database(path)

In [ ]:
import os
from pathlib import Path
from abc import ABC, abstractmethod

class Feature(ABC):
    
    @abstractmethod
    def add_to_database(self):
        pass
    

class CellRecording(Feature):
    
    def add_to_database(self):
        pass
    
    
class Subdirectories:
    
    def __init__(self, root_dir: Path):
        self.root_dir = root_dir
        self.create_missing_subdirectories()
        self.assign_subdirectories_as_attributes()
        
    def create_missing_subdirectories(self):
        # check for each element in a list of subdirs, whether they exist --> create if not
        pass
    
    def assign_subdirectories_as_attributes(self):
        # use list of subdirs and set the path to each as attribute
        pass   


class Database:
    
    """
    The database is supposed to hold all information about all recorded cells that were added to it.
    These information exclude the raw data (only contain filepaths to raw data),
    but include general metadata like celltype, stimulus type, brain region, pharamacological treatment.
    Potentially, some intermediate data could be added (like how the cell reacted upon stimulation, see dashboard).
    Overall, the database should allow the selection of cells based on mix-and-match criteria 
    for further (statistical) analyses and plotting.
    """
    
    def __init__(self, root_dir: Path):
        self.root_dir = root_dir
        self.subdirectories = Subdirectories(root_dir = root_dir)

    def load_database_from_disk(self):
        # Option to load previously created and saved database
        pass
    
    def save_database_to_disk(self):
        # Save all information to disk
        pass
    
    def add_new_feature(self, new_feature: Feature):
        # By providing the Path to a new directory, the user has the option to add new recordings to the database
        # should trigger the database to update itself (e.g. categories dictionary)
        pass


    

        
